In [1]:
import os
import numpy as np
import pandas as pd

In [6]:
### still going to need to make new dir for copies ###

data_dir = "./Data/EMG_data_for_gestures-master/"
subj_nums = os.listdir(data_dir)
subj_data = {n : os.listdir(data_dir + n) for n in subj_nums}

In [7]:
for n in subj_nums:
    print(os.listdir(data_dir + n + "/"))

['2_raw_data_09-50_21.03.16.txt', '1_raw_data_09-49_21.03.16.txt']
['1_raw_data_10-38_11.04.16.txt', '2_raw_data_10-40_11.04.16.txt']
['2_raw_data_12-43_23.03.16.txt', '1_raw_data_12-41_23.03.16.txt']
['2_raw_data_09-34_11.04.16.txt', '1_raw_data_09-32_11.04.16.txt']
['1_raw_data_10-28_30.03.16.txt', '2_raw_data_10-29_30.03.16.txt']
['1_raw_data_20-28_24.04.16.txt', '2_raw_data_20-30_24.04.16.txt']
['2_raw_data_11-20_23.03.16.txt', '1_raw_data_11-19_23.03.16.txt']
['2_raw_data_13-13_22.03.16.txt', '1_raw_data_13-12_22.03.16.txt']
['2_raw_data_12-14_25.04.16.txt', '1_raw_data_12-12_25.04.16.txt']
['2_raw_data_10-23_29.03.16.txt', '1_raw_data_10-22_29.03.16.txt']
['2_raw_data_12-06_27.04.16.txt', '1_raw_data_12-04_27.04.16.txt']
['2_raw_data_18-03_24.04.16.txt', '1_raw_data_18-02_24.04.16.txt']
['2_raw_data_12-16_23.03.16.txt', '1_raw_data_12-14_23.03.16.txt']
['2_raw_data_12-20_06.04.16.txt', '1_raw_data_12-19_06.04.16.txt']
['1_raw_data_09-50_15.04.16.txt', '2_raw_data_09-51_15.04.16.t

In [8]:
# looking at this subject
tst_subj = "03"

In [9]:
tst_path_0 = data_dir + tst_subj + "/" + subj_data[tst_subj][0]
tt0 = pd.read_table(tst_path_0)

In [10]:
tst_path_1 = data_dir + tst_subj + "/" + subj_data[tst_subj][1]
tt1 = pd.read_table(tst_path_1)

In [12]:
tst_path_1 # example path

'./Data/EMG_data_for_gestures-master/03/1_raw_data_09-32_11.04.16.txt'

In [13]:
# rename "time" variable to "time_ms"
# rename "class" variable to "gesture"
colnames = list(tt0.columns)
colnames[0] = "time_ms"
colnames[-1] = "gesture"
tt0.columns = colnames
tt1.columns = colnames

In [14]:
# unique gestures in both files
ug0 = tt0.gesture.unique()
ug1 = tt1.gesture.unique()

In [34]:
tt0.iloc[tt0.index[:brk_pt]]

,time_ms,channel1,channel2,channel3,channel4,channel5,channel6,channel7,channel8,gesture
0,1,-0.00004,-0.00002,-0.00001,-0.00002,-0.00001,-0.00001,0.00002,-0.00001,0
1,4,-0.00004,-0.00002,-0.00001,-0.00002,-0.00001,-0.00001,0.00002,-0.00001,0
2,5,-0.00004,-0.00002,-0.00001,-0.00002,-0.00001,-0.00001,0.00002,-0.00001,0
3,6,-0.00004,-0.00002,-0.00001,-0.00002,-0.00001,-0.00001,0.00002,-0.00001,0
4,7,-0.00002,-0.00001,0.00000,0.00001,0.00000,-0.00001,-0.00002,0.00001,0
5,8,0.00000,0.00000,-0.00002,-0.00002,-0.00002,-0.00001,-0.00001,-0.00001,0
6,9,0.00000,0.00000,-0.00002,-0.00002,-0.00002,-0.00001,-0.00001,-0.00001,0
7,10,0.00000,0.00000,-0.00002,-0.00002,-0.00002,-0.00001,-0.00001,-0.00001,0
8,11,0.00000,0.00000,-0.00002,-0.00002,-0.00002,-0.00001,-0.00001,-0.00001,0
9,12,0.00000,0.00000,-0.00002,-0.00002,-0.00002,-0.00001,-0.00001,-0.00001,0


In [35]:
def find_break_pts(gesture, t = 1000):
    """find and return index for moments in data frame with > t ms between pt and next pt"""
    nrow = gesture.shape[0]
    brk_pts = [i-1 for i in range(2, nrow) if gesture.iloc[i].time_ms - gesture.iloc[i-1].time_ms > t]
    return brk_pts

In [36]:
find_break_pts(tt0[tt0.gesture == ug0[4]])

[1736]

In [28]:
def gest_dict(filepath):
    """iterate through file and return dict where key is gesture # and value is DataFrame list"""
    df = pd.read_table(filepath)
    gests = df.gesture.unique()
    all_gst = dict()
    # seperate gestures in file
    for g in gests:
        brk_pts = find_break_pts(df[df.gesture == g])
        # split single gesture df into iterations of gesture g
        gd = dict()
        prev_b = 0
        for n, b in enumerate(brk_pts):
            gd[n] = df[df.gesture == g].iloc[prev_b:b]
            prev_b = b
        
    return gd

SyntaxError: invalid syntax (<ipython-input-28-895cf17a8922>, line 8)

In [16]:
brk_pt

1545

In [17]:
for j in subj_nums:
    # get files 1 and 2 for subject j
    fp0 = data_dir + j + "/" + subj_data[j][0]
    fp1 = data_dir + j + "/" + subj_data[j][1]
    gest_dict(fp0)

['30',
 '06',
 '09',
 '03',
 '05',
 '21',
 '17',
 '01',
 '16',
 '26',
 '32',
 '04',
 '08',
 '27',
 '14',
 '23',
 '25',
 '18',
 '29',
 '15',
 '28',
 '12',
 '19',
 '07',
 '20',
 '31',
 '13',
 '36',
 '24',
 '33',
 '11',
 '35',
 '22',
 '34',
 '10',
 '02']

In [ ]:
# check if residual movement from gesture persists